# Combine transitions into one file

Used to be one, now there are many and by customers

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import datetime
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [2]:
TODAY = datetime.date.today().strftime("%d%m%y")
print(TODAY)

071221


In [3]:
DT_IN='20211206'

### National transitions

In [4]:
#ht_file = glob.glob('../data/raw/{}/Transitions/*Hard Transition.txt'.format(DT_IN))[0]
# Load transition data and format it for merge
#transitions = pd.read_csv(ht_file, sep='\t',usecols=['Cur. UCC14', 'Pr. UCC14', 'Valid From','UCC14 Description'])


In [5]:

# transitions = transitions.append(pd.read_excel('../data/raw/{}/Transitions/Manual_transitions.xlsx'.format(DT_IN), 
#                                                sheet_name = 'national',usecols=['Cur. UCC14', 'Pr. UCC14', 'Valid From']))
transitions = pd.read_excel('../data/raw/{}/Transitions/Manual_transitions.xlsx'.format(DT_IN), 
                                               sheet_name = 'national',usecols=['Cur. UCC14', 'Pr. UCC14', 'Valid From'])

transitions['First_G_WEEK'] = pd.to_datetime(transitions['Valid From'],format='%m-%d-%Y').dt.strftime('W%Y%U')

transitions['Cur. UCC14'] = transitions['Cur. UCC14'].astype(str).str.zfill(14)
transitions['Pr. UCC14'] = transitions['Pr. UCC14'].astype(str).str.zfill(14)

#del transitions['Valid From']

In [6]:
#ipks = transitions[transitions['UCC14 Description'].fillna('').str.contains('IPK')]['Cur. UCC14'].unique()

In [7]:
# Append Unit Per Case correction coefficient


In [8]:
sales = pd.read_csv(r'../data/raw/{}/DOMO_UPC_PLANS EXPORT.csv'.format(DT_IN), usecols=['PAD'],dtype=str)

In [9]:
sales = sales.drop_duplicates()

In [10]:
sales = sales.rename(columns = {'PAD':'PLANNED_ACCOUNT_CD'})

In [11]:
ch = pd.read_excel('../data/raw/{}/DHUB CH Extract.xlsx'.format(DT_IN),dtype=str)

C:\Users\bogda\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
ch['PLANNED_ACCOUNT_CD'] = ch['CUST_ID_NR_CD'] 
ch['Customer Name'] = ch['APG'] 

ch = ch[['PLANNED_ACCOUNT_CD','Customer Name']].drop_duplicates(subset = 'PLANNED_ACCOUNT_CD')

In [13]:
sales_apg = sales.merge(ch)

In [14]:
transitions = transitions.assign(key=1).merge(sales.assign(key=1), how='outer', on='key')
del transitions['key']

In [15]:
transitions.columns

Index(['Valid From', 'Pr. UCC14', 'Cur. UCC14', 'First_G_WEEK',
       'PLANNED_ACCOUNT_CD'],
      dtype='object')

In [16]:
#transitions = transitions[(transitions.PLANNED_ACCOUNT_CD == '000174')|(~transitions['Cur. UCC14'].isin(ipks))]

In [17]:
transitions[transitions['Cur. UCC14'] == '10079400485295']

,Valid From,Pr. UCC14,Cur. UCC14,First_G_WEEK,PLANNED_ACCOUNT_CD


### transitions by apg

In [18]:
ht_file = glob.glob('../data/raw/{}/Transitions/Current Transitions *'.format(DT_IN))[0]
ht_file

'../data/raw/20211206/Transitions\\Current Transitions 11.19.xlsx'

In [19]:
hard_trans = pd.read_excel(ht_file,header = 1)
hard_trans['First_G_WEEK'] = pd.to_datetime(hard_trans['Effective In Date']).dt.strftime('W%Y%U')
hard_trans['Cur. UCC14'] = hard_trans['Forecast Unit.1'].astype(str).str.zfill(14)
hard_trans['Pr. UCC14'] = hard_trans['Forecast Unit'].astype(str).str.zfill(14)


In [20]:
hard_trans = hard_trans[(~hard_trans['Part Description.1'].str.contains('IPK'))|(hard_trans['Part Description.1'] == 'WALMART')]     

In [21]:
hard_trans = hard_trans.merge(sales_apg)
hard_trans = hard_trans[['Cur. UCC14','Pr. UCC14','First_G_WEEK','PLANNED_ACCOUNT_CD']].drop_duplicates()

In [22]:
hard_trans

,Cur. UCC14,Pr. UCC14,First_G_WEEK,PLANNED_ACCOUNT_CD
0,10764302018213,10764302215599,W202117,000691
1,10764302018152,10764302231056,W202122,000691
2,10764302017834,10764302190360,W202115,000691
5,10764302019975,10764302017025,W202116,000691
7,10764302018251,10764302280207,W202118,000691
...,...,...,...,...
145352,10732913450265,10732913450012,W202109,000488
145353,10011111034874,10011111002002,W202148,000488
145354,10011111034898,10011111002026,W202148,000488
145355,10011111034867,10011111016955,W202148,000488


In [23]:
transitions = hard_trans.append(transitions)

### Customer transitions

In [24]:
ct_file = glob.glob('../data/raw/{}/Transitions/*Customer*Transition.xlsx'.format(DT_IN))[0]

In [25]:
cust_trans = pd.read_excel(ct_file,dtype=str)

In [26]:
cust_trans = cust_trans.append(pd.read_excel('../data/raw/{}/Transitions/Transition Log.xlsx'.format(DT_IN)))

In [27]:
cust_trans = cust_trans[~cust_trans['Old UCC 14'].isna()]

In [28]:
cust_trans.shape

(1092, 23)

In [29]:
cust_trans['PLANNED_ACCOUNT_CD'] = cust_trans['Planned Account'].str[:6]
cust_trans['Cur. UCC14'] = cust_trans['New UCC 14'].astype('int64').astype(str).str.zfill(14)
cust_trans['Pr. UCC14'] = cust_trans['Old UCC 14'].astype('int64').astype(str).str.zfill(14)
cust_trans['First_G_WEEK'] = cust_trans['transition date'].astype('datetime64[ns]').dt.strftime('W%Y%U')



In [30]:
cust_trans[cust_trans['Pr. UCC14'].astype(str).str.contains('10079400482195')]

,APG,Division,Planned Account,Promo Family,UPC,Old UCC 14,New UCC 14,year,transition week,transition date,case pack change,sales history to be moved,reset date,Request Date,PlannedAccount,Requester,Processed Date,Processed by ML,Comments,Processed by,Reprocessed date,Unnamed: 11,Key,PLANNED_ACCOUNT_CD,Cur. UCC14,Pr. UCC14,First_G_WEEK
397,NaN,NaN,000235-000000 : DOLLAR GENERAL,NaN,NaN,10079400482195,10079400494419,NaN,NaN,12/20/2021,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,000235,10079400494419,10079400482195,W202151
406,NaN,NaN,000239-000000 : FAMILY DOLLAR,NaN,NaN,10079400482195,10079400494419,NaN,NaN,12/20/2021,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,000239,10079400494419,10079400482195,W202151
360,NaN,NaN,NaN,NaN,NaN,10079400482195,10079400494419,NaN,NaN,2021-12-20 00:00:00,NaN,NaN,NaN,2021-09-01,000235-000000:DOLLARGENERAL,Matthew McGurgan,NaT,NaN,NaN,NaN,NaN,NaN,000235-000000:DOLLARGENERAL1007940048219510079...,NaN,10079400494419,10079400482195,W202151
369,NaN,NaN,NaN,NaN,NaN,10079400482195,10079400494419,NaN,NaN,2021-12-20 00:00:00,NaN,NaN,NaN,2021-09-01,000239-000000:FAMILYDOLLAR,Matthew McGurgan,NaT,NaN,NaN,NaN,NaN,NaN,000239-000000:FAMILYDOLLAR10079400482195100794...,NaN,10079400494419,10079400482195,W202151


In [31]:
cust_trans = cust_trans[['Cur. UCC14', 'Pr. UCC14', 'First_G_WEEK', 'PLANNED_ACCOUNT_CD']]

In [32]:
cust_trans = cust_trans.drop_duplicates(subset = ['Cur. UCC14', 'Pr. UCC14',  'PLANNED_ACCOUNT_CD'])

In [33]:
cust_trans.shape

(473, 4)

### Hierarchy

In [34]:
transitions = transitions.append(cust_trans)

In [35]:
transitions[transitions['Pr. UCC14'].astype(str).str.contains('10079400482195')]

,Cur. UCC14,Pr. UCC14,First_G_WEEK,PLANNED_ACCOUNT_CD,Valid From
76611,10079400494419,10079400482195,W202205,000004,NaN
76612,10079400494419,10079400482195,W202205,000010,NaN
76613,10079400494419,10079400482195,W202205,000020,NaN
76614,10079400494419,10079400482195,W202205,000065,NaN
76615,10079400494419,10079400482195,W202205,000074,NaN
...,...,...,...,...,...
76851,10079400494419,10079400482195,W202205,003462,NaN
76852,10079400494419,10079400482195,W202205,003466,NaN
397,10079400494419,10079400482195,W202151,000235,NaN
406,10079400494419,10079400482195,W202151,000239,NaN


In [36]:
ph_ucc = pd.read_excel('../data/master_data/DHUB PH Extract.xlsx' ,usecols = ['Find CaseUPC','UCC 14 Code'])     

C:\Users\bogda\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [37]:
ph_ucc['UCC 14 Code'] = ph_ucc['UCC 14 Code'].astype(str).str.zfill(14)

In [38]:
transitions = pd.merge(transitions, ph_ucc, how='left', left_on='Pr. UCC14', right_on='UCC 14 Code'
                       ).drop(columns=['UCC 14 Code']).rename(columns={'Find CaseUPC': 'UPC_OLD'})
transitions = pd.merge(transitions, ph_ucc, how='left', left_on='Cur. UCC14', right_on='UCC 14 Code'
                       ).drop(columns=['UCC 14 Code']).rename(columns={'Find CaseUPC': 'UPC_NEW'})

In [39]:
#transitions = transitions[transitions.UPC_OLD != transitions.UPC_NEW]
transitions = transitions[~(transitions.UPC_NEW.isna()|transitions.UPC_OLD.isna()|transitions.PLANNED_ACCOUNT_CD.isna())]

In [40]:
transitions = transitions[['PLANNED_ACCOUNT_CD','First_G_WEEK', 'UPC_OLD', 'UPC_NEW']].drop_duplicates(subset=['PLANNED_ACCOUNT_CD','UPC_OLD'],keep = 'last')


In [41]:
transitions['pa_upc_old'] = transitions.PLANNED_ACCOUNT_CD + '-' + transitions.UPC_OLD
transitions['pa_upc_new'] = transitions.PLANNED_ACCOUNT_CD + '-' + transitions.UPC_NEW

In [42]:
    newest = transitions[['pa_upc_old','pa_upc_new','First_G_WEEK']].drop_duplicates()

    newest = newest[newest['pa_upc_old'].isin(newest['pa_upc_new'])]

    newest.columns = ['pa_upc_new','pa_upc_newest','newest_week']

    newest = newest[newest.pa_upc_new!=newest.pa_upc_newest]
    newest.columns = ['pa_upc_new','pa_upc_newest','newest_week']


In [43]:
newest

,pa_upc_new,pa_upc_newest,newest_week
1025,000004-007940045329,000004-007940048804,W202118
1026,000010-007940045329,000010-007940048804,W202118
1027,000020-007940045329,000020-007940048804,W202118
1028,000065-007940045329,000065-007940048804,W202118
1029,000074-007940045329,000074-007940048804,W202118
...,...,...,...
66972,000668-073291345025,000668-073291345070,W202209
66988,000174-073291345024,000174-073291345072,W202209
66997,000174-001111102788,000174-001111102517,W202126
66998,000174-001111102795,000174-001111102518,W202126


In [44]:
transitions_check = transitions.merge(newest,how='left')

In [46]:
while True:
    
    newest = transitions[['pa_upc_old','pa_upc_new','First_G_WEEK']].drop_duplicates()

    newest = newest[newest['pa_upc_old'].isin(newest['pa_upc_new'])]

    newest.columns = ['pa_upc_new','pa_upc_newest','newest_week']

    newest = newest[newest.pa_upc_new!=newest.pa_upc_newest]
    newest.columns = ['pa_upc_new','pa_upc_newest','newest_week']
    transitions = transitions.merge(newest,how='left')
    if transitions[transitions.newest_week>transitions.First_G_WEEK].shape[0] == 0:
        break
    transitions['pa_upc_new'] = np.where(transitions.newest_week>transitions.First_G_WEEK,transitions['pa_upc_newest'],transitions['pa_upc_new'])
    
    del transitions['pa_upc_newest']
    del transitions['newest_week']
    print ('new cycle')

new cycle


In [47]:
del transitions['pa_upc_newest']
del transitions['newest_week']

In [48]:
transitions['UPC_OLD'] = transitions.pa_upc_old.str[-12:]
transitions['UPC_NEW'] = transitions.pa_upc_new.str[-12:]

In [49]:
del transitions['pa_upc_old']
del transitions['pa_upc_new']

In [50]:
md = pd.read_csv(r'../data/raw/{}/DOMO_PRODUCT_HIERARCHY EXPORT.csv'.format(DT_IN), usecols=['CS_UPC_NR_CD', 'UNITS_PER_CASE'])
md['UPC'] = md['CS_UPC_NR_CD'].astype(str).str.zfill(12)
md = md[['UPC', 'UNITS_PER_CASE']]
transitions = pd.merge(transitions, md, how='left', left_on='UPC_OLD', right_on='UPC').drop(columns=['UPC']).rename(columns={'UNITS_PER_CASE': 'U_per_CS_OLD'})
transitions = pd.merge(transitions, md, how='left', left_on='UPC_NEW', right_on='UPC').drop(columns=['UPC']).rename(columns={'UNITS_PER_CASE': 'U_per_CS_NEW'})
transitions['UPC_CORR_COEF'] = transitions['U_per_CS_OLD'] / transitions['U_per_CS_NEW']
transitions['UPC_CORR_COEF'] = transitions['UPC_CORR_COEF'].fillna(1)


In [51]:
transitions = transitions[['PLANNED_ACCOUNT_CD','UPC_OLD', 'UPC_NEW', 'UPC_CORR_COEF','First_G_WEEK'
                          ]].drop_duplicates(subset=['PLANNED_ACCOUNT_CD','UPC_OLD'],keep = 'last')

In [52]:
transitions[transitions['UPC_OLD'].astype(str).str.contains('007940048033')]

,PLANNED_ACCOUNT_CD,UPC_OLD,UPC_NEW,UPC_CORR_COEF,First_G_WEEK
10,000691,007940048033,007940048697,1.0,W202144
10670,000004,007940048033,007940048697,1.0,W202144
10671,000010,007940048033,007940048697,1.0,W202144
10672,000020,007940048033,007940048697,1.0,W202144
10673,000065,007940048033,007940048697,1.0,W202144
...,...,...,...,...,...
20445,003030,007940048033,007940048697,1.0,W202144
20446,003463,007940048033,007940048697,1.0,W202144
20813,000488,007940048033,007940048697,1.0,W202144
45642,000235,007940048033,007940048697,1.0,W202151


In [53]:
transitions.to_csv('../data/master_data/Transitions_combined.txt',index=False)

In [54]:
transitions.to_csv('../data/raw/{}/Transitions/Transitions_combined.txt'.format(DT_IN),index=False)

In [55]:
transitions.to_csv('../data/transitions/Transitions_combined'+TODAY+'.txt',index=False)

In [ ]:
stop

In [56]:
    transitions = pd.read_csv('../data/master_data/Transitions_combined.txt',
                              dtype={'PLANNED_ACCOUNT_CD':'str','UPC_OLD':'str','UPC_NEW':'str','UPC_CORR_COEF':'float','First_G_WEEK':'str'})
    transitions = transitions.rename(columns = {'PLANNED_ACCOUNT_CD':'CUSTOMER'})

In [58]:
transitions.assign(temp_merge=1)

,CUSTOMER,UPC_OLD,UPC_NEW,UPC_CORR_COEF,First_G_WEEK,temp_f
0,000691,076430201702,076430201997,1.333333,W202116,1
1,000691,001111100793,001111103044,1.333333,W202119,1
2,000691,007704300236,007704300345,1.000000,W202125,1
3,000691,007940040358,007940048023,1.000000,W202104,1
4,000691,007940040411,007940048022,1.000000,W202104,1
...,...,...,...,...,...,...
45655,000239,007940067027,007940048942,1.000000,W202140,1
45656,000239,007940034370,007940048943,1.000000,W202140,1
45657,000239,007940058956,007940048940,1.000000,W202141,1
45658,000239,007940034308,007940048945,1.000000,W202142,1
